<a href="https://colab.research.google.com/github/aaronjoel/DeepUnderstandingOfDeepLearning/blob/main/chapter_11_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install lightning

In [2]:
import torch ## torch let's us create tensors and also provides helper functions
import torch.nn as nn ## torch.nn gives us nn.Module(), nn.Embedding() and nn.Linear()
import torch.nn.functional as F # This gives us the softmax() and argmax()
from torch.optim import Adam ## We will use the Adam optimizer, which is essentially
                             ## a slightly less stochastic version of stochastic gradient descent.

from torch.utils.data import TensorDataset, DataLoader ## We'll store our data in DataLoaders
import lightning as L

# Create the datasets that we will use for training Encoder-Decoder model

To make the model at least a little bit interesting, we will translate two english phrases, __Let's go__ and __to go__ into spanish. __Let's go__ should translate to __vamos <EOS>__ and __to go__ should translate to __ir <EOS>__.

In [3]:
## first, we create a dictionary that maps vocabulary tokens to id numbers...
english_token_to_id = {
    'lets': 0,
    'to': 1,
    'go': 2,
    '<EOS>': 3   ## <EOS> = end of sequence
}

## ...then we create a dictionary that maps the ids to tokens. This will help us interpret the out
## We use the "map()" function to apply the "reversed()" function to each tuple (i.e. ('lets', 0)) stored
## in the token_to_id dictionary. We then use dict() to make a new dictionary from the reversed tuples.
english_id_to_token = dict(map(reversed, english_token_to_id.items()))

spanish_token_to_id = {
    'ir': 0,
    'vamos': 1,
    'y': 2,
    '<EOS>': 3
}

spanish_id_to_token = dict(map(reversed, spanish_token_to_id.items()))

inputs = torch.tensor([[english_token_to_id["lets"],
                        english_token_to_id["go"]],

                       [english_token_to_id["to"],
                        english_token_to_id["go"]]])

labels = torch.tensor([[spanish_token_to_id["vamos"],
                        spanish_token_to_id["<EOS>"]],

                       [spanish_token_to_id["ir"],
                        spanish_token_to_id["<EOS>"]]])

In [4]:
english_token_to_id

{'lets': 0, 'to': 1, 'go': 2, '<EOS>': 3}

In [5]:
english_id_to_token

{0: 'lets', 1: 'to', 2: 'go', 3: '<EOS>'}

In [6]:
spanish_id_to_token

{0: 'ir', 1: 'vamos', 2: 'y', 3: '<EOS>'}

In [7]:
spanish_token_to_id

{'ir': 0, 'vamos': 1, 'y': 2, '<EOS>': 3}

In [8]:
inputs

tensor([[0, 2],
        [1, 2]])

In [9]:
labels

tensor([[1, 3],
        [0, 3]])

Now that we have created the data that we want to train the embeddings with, we'll store it in a `DataLoader`. Since our dataset is so small, using a `DataLoader` is a little bit of an overkill, but it's easy to do, and it will allow us to easily scale up to a much larger vocabulary when the time comes.

In [10]:
## Now let's package everything up into a DataLoader...
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [11]:
class seq2seq_attention(L.LightningModule):

  def __init__(self, max_len=2):
    super().__init__()
    self.max_decoder_length = max_len
    L.seed_everything(seed=42)

    ## ENCODING
    self.encoder_we = nn.Embedding(num_embeddings=4, # num_embeddings = # of tokens in input vocabulary
                                   embedding_dim=1)  # embedding_dim = # of embedding values per token

    self.encoder_lstm = nn.LSTM(input_size=1, # encoder_size = number of inputs
                                hidden_size=1, # hidden_size = number of outputs
                                num_layers=1)  # num_layers = how many lstm's to stack

    ## DECODING
    self.decoder_we = nn.Embedding(num_embeddings=4, embedding_dim=1)

    self.decoder_lstm = nn.LSTM(input_size=1, hidden_size=1, num_layers=1)

    self.decoder_fc = nn.Linear(in_features=2, # in_features = number of outputs per lstm * 2 (encoder + decoder)
                                out_features=4) # out_features = number of words in the output vocabulary

    self.loss = nn.CrossEntropyLoss()

  def forward(self, input, output=None):
    #outputs = torch.zeros(self.max_decoder_length, 4)

    ## ENCODING

    ## first, use the encoder stage to create an intermediate encoding of the input text
    encoder_embeddings = self.encoder_we(input)
    encoder_lstm_output, (encoder_lstm_hidden, encoder_lstm_cell) = self.encoder_lstm(encoder_embeddings)

    ## DECODING

    ## We start by initializing the decoder with the <EOS> token...
    decoder_token_id = torch.tensor([spanish_token_to_id["<EOS>"]])
    decoder_embeddings = self.decoder_we(decoder_token_id)
    decoder_lstm_output, (decoder_lstm_hidden, decoder_lstm_cell) = self.decoder_lstm(decoder_embeddings,
                                                                                      (encoder_lstm_hidden,
                                                                                       encoder_lstm_cell))

    ## Calculate attention here (using unnormalised cosine similarity, aka, dot product, to score the alignment)
    ## In other words, we're trying to find words in theinput that are used in the same context as those
    ## proposed by the lstm.
    sims = torch.matmul(decoder_lstm_output, encoder_lstm_output.transpose(dim0=0, dim1=1))

    ## Apply softmax to determine what percent of each token's value to use in the final attention values.
    attention_percents = F.softmax(sims, dim=1)

    ## Scale the values by their associated percentages and add them up.
    attention_values = torch.matmul(attention_percents, encoder_lstm_output)

    ## Lastly, we need to concatenate the attention values with the short term
    ## memories from the first decoder lstm
    values_to_fc_layer = torch.cat((attention_values, decoder_lstm_output), 1)

    output_values = self.decoder_fc(values_to_fc_layer)

    outputs = output_values
    predicted_id = torch.tensor([torch.argmax(output_values)])
    predicted_ids = predicted_id

    for i in range(1, self.max_decoder_length):
      if output == None:
        if predicted_id == spanish_token_to_id["<EOS>"]:
          break
        decoder_embeddings = self.decoder_we(predicted_id)
      else:
        decoder_embeddings = self.decoder_we(torch.tensor([output[i-1]]))

      decoder_lstm_output, (decoder_lstm_hidden, decoder_lstm_cell) = self.decoder_lstm(decoder_embeddings,
                                                                                        (decoder_lstm_hidden, decoder_lstm_cell))

      sims = torch.matmul(decoder_lstm_output, encoder_lstm_output.transpose(dim0=0, dim1=1))
      ## Apply softmax to determine what percent of each token's value
      ## to use in the final attention values.
      attention_percents = F.softmax(sims, dim=1)

      ## Scale the values by their associated percentages and add them up
      attention_values = torch.matmul(attention_percents, encoder_lstm_output)

      values_to_fc_layer = torch.cat((attention_values, decoder_lstm_output), 1)

      output_values = self.decoder_fc(values_to_fc_layer)
      #outputs[i] = output_values
      outputs = torch.cat((outputs, output_values), 0)
      predicted_id = torch.tensor([torch.argmax(output_values)])
      predicted_ids = torch.cat((predicted_ids, predicted_id))

    return outputs

  def configure_optimizers(self):
    return Adam(self.parameters(), lr=0.1)

  def training_step(self, batch, batch_idx):
    input_tokens, labels = batch # collect input
    output = self.forward(input_tokens[0], labels[0])
    loss = self.loss(output, labels[0])

    ## Logging the loss
    # self.log("train_loss", loss)

    return loss

In [12]:
model = seq2seq_attention()
outputs = model.forward(input=torch.tensor([english_token_to_id["lets"],
                                            english_token_to_id["go"]]), ## translate "lets go", we should get "vamos <EOS>"
                        output=None)

print("Translated text:")
predicted_ids = torch.argmax(outputs, dim=1)
for id in predicted_ids:
  print("\t", spanish_id_to_token[id.item()])

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Translated text:
	 y
	 y


In [13]:
trainer = L.Trainer(max_epochs=20, accelerator='cpu')
trainer.fit(model, train_dataloaders=dataloader)

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | encoder_we   | Embedding        | 4      | train
1 | encoder_lstm | LSTM             | 16     | train
2 | decoder_we   | Embedding        | 4  

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [14]:
outputs = model.forward(input=torch.tensor([english_token_to_id["lets"],
                                            english_token_to_id["go"]]), ## translate "lets go", we should get "vamos <EOS>"
                        output=None)

print("Translated text:")
predicted_ids = torch.argmax(outputs, dim=1)
for id in predicted_ids:
    print("\t", spanish_id_to_token[id.item()])

Translated text:
	 vamos
	 <EOS>


In [15]:
outputs = model.forward(input=torch.tensor([english_token_to_id["to"],
                                            english_token_to_id["go"]]), ## translate "to go", we should get "ir <EOS>"
                        output=None)

print("Translated text:")
predicted_ids = torch.argmax(outputs, dim=1)
for id in predicted_ids:
    print("\t", spanish_id_to_token[id.item()])

Translated text:
	 ir
	 <EOS>


In [16]:
print("After optimization, the parameters are...")
for name, param in model.named_parameters():
    print(name, param.data)

After optimization, the parameters are...
encoder_we.weight tensor([[-1.7787],
        [ 2.0923],
        [-0.1636],
        [ 0.2303]])
encoder_lstm.weight_ih_l0 tensor([[-1.6984],
        [-2.2462],
        [ 1.4013],
        [-1.5993]])
encoder_lstm.weight_hh_l0 tensor([[-0.7744],
        [-1.1266],
        [ 1.5920],
        [-0.4829]])
encoder_lstm.bias_ih_l0 tensor([ 1.8437,  1.4772, -1.0555, -0.0629])
encoder_lstm.bias_hh_l0 tensor([ 0.6121,  1.2121, -0.9948,  0.3456])
decoder_we.weight tensor([[-0.8066],
        [-2.2045],
        [-0.9890],
        [ 2.8455]])
decoder_lstm.weight_ih_l0 tensor([[-0.4647],
        [ 2.1268],
        [-1.5542],
        [ 0.4553]])
decoder_lstm.weight_hh_l0 tensor([[-1.3270],
        [ 1.8276],
        [-0.5032],
        [-1.9216]])
decoder_lstm.bias_ih_l0 tensor([ 1.8560, -0.7807,  0.7111,  3.0293])
decoder_lstm.bias_hh_l0 tensor([ 2.1653, -0.9467,  0.8675,  2.3154])
decoder_fc.weight tensor([[ 0.4913, -1.7664],
        [-1.1043, -1.1885],
      